In [1]:
str ="Python语言程序设计"
str.isnumeric()

False

In [2]:
s1 ="袋鼠"

print("{0}生活在主要由母{0}和小{0}组成的较小的群体里。".format(s1))

袋鼠生活在主要由母袋鼠和小袋鼠组成的较小的群体里。


In [3]:
s1 ="企鹅"

s2 ="超级游泳健将"

print("{0:^4}:{1:!<9}".format(s1,s2))

 企鹅 :超级游泳健将!!!


In [4]:
x = “foo”  

y = 2  

print(x+y) 

SyntaxError: invalid character '“' (U+201C) (3021879201.py, line 1)

In [5]:
s = 'Python is beautiful!'

print(s[0:6])

Python


In [5]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from scipy.stats import randint
from tqdm import tqdm

# 加载数据
file_path = 'E:\\竞赛\\服创赛-A08\\data\\欺诈手段特征集.xlsx'
data = pd.read_excel(file_path)

# 分离特征和目标变量
X = data.drop(columns='RES')
y = data['RES']

# AUC评分函数
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

# 十折交叉验证
cv = StratifiedKFold(n_splits=10)

# 超参数空间
param_dist = {
    'n_estimators': randint(64, 228),
    'max_depth': [4, 5,6,7,8,9,10],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

# 初始化随机森林分类器
rf = RandomForestClassifier(bootstrap=True, max_features='sqrt', class_weight=None, random_state=123, n_jobs=-1)

# 存储所有AUC > 0.95的参数组合及其AUC
successful_params = []

# 使用tqdm进行进度条显示
for i in tqdm(range(300)):
    random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=1, scoring=auc_scorer, cv=cv, random_state=123+i)
    random_search.fit(X, y)

    # 检查AUC是否大于0.95
    if random_search.best_score_ > 0.95:
        successful_params.append((random_search.best_params_, random_search.best_score_))

# 输出所有AUC > 0.95的参数组合及其AUC
if successful_params:
    for params, score in successful_params:
        print("Parameters:", params)
        print("AUC Score:", score)
        print("---")
else:
    print("No configuration found with AUC > 0.95")



D:\Anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [44:25<00:00,  8.89s/it]

No configuration found with AUC > 0.95


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import train_test_split

# Prepare your dataset
X = data.drop('RES', axis=1)
y = data['RES']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Define the parameter grid
param_grid_modified = {
    'n_estimators': list(range(50, 151)),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11))
}

# Create a RandomForestClassifier model
model = RandomForestClassifier(bootstrap=True, class_weight=None, random_state=123, n_jobs=-1)

# Define the scorer for AUC
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

# Create and perform the grid search
grid_search_modified = GridSearchCV(model, param_grid_modified, cv=10, scoring=auc_scorer, n_jobs=-1)
grid_search_modified.fit(X_train, y_train)

# Extract the best parameters and score
best_params = grid_search_modified.best_params_
best_score = grid_search_modified.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


D:\Anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [4]:
# This script needs these libraries to be installed: 
#   numpy, xgboost

import wandb
from wandb.xgboost import WandbCallback

import numpy as np
import xgboost as xgb


# setup parameters for xgboost
param = {
    "objective" : "multi:softmax",
    "eta" : 0.1,
    "max_depth": 6,
    "nthread" : 4,
    "num_class" : 6
}

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config=param
)

# download data from wandb Artifacts and prep data
wandb.use_artifact('wandb/intro/dermatology_data:v0', type='dataset').download('.')
data = np.loadtxt(
    "./dermatology.data", delimiter=",",
    converters={33: lambda x: int(x == "?"), 34: lambda x: int(x) - 1},
)
sz = data.shape

train = data[: int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7) :, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
watchlist = [(xg_train, "train"), (xg_test, "test")]

# add another config to the wandb run
num_round = 5
wandb.config["num_round"] = 5
wandb.config["data_shape"] = sz

# pass WandbCallback to the booster to log its configs and metrics
bst = xgb.train(
    param, xg_train, num_round, evals=watchlist, 
    callbacks=[WandbCallback()]
)

# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]

# log your test metric to wandb 
wandb.summary["Error Rate"] = error_rate

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()
      

wandb: Currently logged in as: zmb1227. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


[0]	train-mlogloss:1.54662	test-mlogloss:1.57447
[1]	train-mlogloss:1.35498	test-mlogloss:1.39797
[2]	train-mlogloss:1.19883	test-mlogloss:1.25218
[3]	train-mlogloss:1.06734	test-mlogloss:1.13098
[4]	train-mlogloss:0.95564	test-mlogloss:1.03268


epoch,▁▃▅▆█
test-mlogloss,█▆▄▂▁
train-mlogloss,█▆▄▂▁
Error Rate,0.09091
epoch,4


In [1]:
import pandas as pd

df = pd.read_csv("E:\竞赛\服创赛-A08\data\process_data.csv")

In [2]:
import job

# 加载模型和特征列表
loaded_data = joblib.load("E:\竞赛\服创赛-A08\model\\lgbm\\lgbm_model_55_0.93191.pkl")
model = loaded_data['model']
features = loaded_data['features']

# 现在，使用加载的特征列表来准备你的输入数据
# 假设 df 是要进行预测的新数据集
df_selected_features = df[features]

NameError: name 'joblib' is not defined